In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

# 전처리 함수: rolling minmax scaling (window=24)
def rolling_minmax_scale(series, window=24):
    roll_min = series.rolling(window=window, min_periods=window).min()
    roll_max = series.rolling(window=window, min_periods=window).max()
    scaled = (series - roll_min) / ((roll_max - roll_min) + 1e-8)
    scaled = scaled.replace([np.inf, -np.inf], np.nan)
    scaled = scaled.fillna(1.0)
    return scaled.clip(upper=1.0)

# 전처리 함수: binning 및 one-hot 인코딩 (OHLC 열, bins=100)
def bin_and_encode(data, features, bins=100, drop_original=True):
    for feature in features:
        data[f'{feature}_Bin'] = pd.cut(data[feature], bins=bins, labels=False)
        one_hot = pd.get_dummies(data[f'{feature}_Bin'], prefix=f'{feature}_Bin').astype(np.int32)
        expected_columns = [f'{feature}_Bin_{i}' for i in range(bins)]
        one_hot = one_hot.reindex(columns=expected_columns, fill_value=0)
        data = pd.concat([data, one_hot], axis=1)
        if drop_original:
            data.drop(columns=[f'{feature}_Bin'], inplace=True)
    numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    for col in numeric_cols:
        data[col] = data[col].astype(np.float32)
    return data

# 분류 모델 (상승/하락 예측) 정의: regression 헤드 제거
class EncoderOnlyTransformerClassifier(nn.Module):
    def __init__(self, input_dim, embedding_dim=512, num_layers=6, nhead=8, 
                 ffn_dim=2048, num_classes=2, max_seq_len=24):
        super(EncoderOnlyTransformerClassifier, self).__init__()
        self.token_embedding = nn.Linear(input_dim, embedding_dim)
        self.position_embedding = nn.Embedding(max_seq_len, embedding_dim)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embedding_dim, nhead=nhead, dim_feedforward=ffn_dim)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(embedding_dim, num_classes)
        
    def forward(self, x):
        batch_size, seq_len, _ = x.shape
        x = self.token_embedding(x)
        positions = torch.arange(seq_len, device=x.device).unsqueeze(0).expand(batch_size, seq_len)
        pos_emb = self.position_embedding(positions)
        x = x + pos_emb
        x = x.transpose(0, 1)  # [seq_len, batch, features]
        x = self.transformer_encoder(x)
        return self.fc(x[-1, :, :])  # 마지막 타임스탭의 출력 사용

# device 및 입력 차원 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_dim = 400  # OHLC 4개 feature를 100 구간으로 binning했으므로

# 분류 모델 생성 후, model_experiment_15.pth의 가중치를 로드
model = EncoderOnlyTransformerClassifier(input_dim=input_dim).to(device)
state_dict = torch.load("model_experiment_15.pth", map_location=device)
model.load_state_dict(state_dict, strict=False)
model.eval()

# CSV 파일에서 시계열 데이터 불러오기 (시간 인덱스가 있다고 가정)
data = pd.read_csv("ETH_upbit_KRW_min5.csv", index_col=0, parse_dates=True)
data = data.sort_index()

# 마지막 시점이 2024-12-18 16:25:00인 부분까지의 데이터에서 마지막 24봉을 입력 시퀀스로 사용
target_last_time = pd.Timestamp("2024-12-18 16:25:00")
if target_last_time not in data.index:
    print("지정한 마지막 시간이 데이터에 없습니다. 데이터의 마지막 행을 사용합니다.")
    target_last_time = data.index[-1]
input_data = data.loc[:target_last_time].tail(24).copy()

# OHLC 열에 대해 전처리 적용
ohlc_features = ['open', 'high', 'low', 'close']
for feature in ohlc_features:
    input_data[feature] = rolling_minmax_scale(input_data[feature], window=24)
input_data_binned = bin_and_encode(input_data.copy(), ohlc_features, bins=100, drop_original=True)
final_input_columns = [col for col in input_data_binned.columns if '_Bin_' in col]
input_seq = input_data_binned[final_input_columns].values  # (24, 400)

# 모델 입력 형태로 변환 (배치 차원 추가)
x = torch.tensor(input_seq, dtype=torch.float32).unsqueeze(0).to(device)

# 예측 수행: 상승/하락 분류 예측
with torch.no_grad():
    output = model(x)
    class_pred = torch.argmax(output, dim=1).item()  # 1: 상승, 0: 하락

trend = "상승" if class_pred == 1 else "하락"
next_candle_time = pd.Timestamp("2024-12-18 16:30:00")
print(f"다음 5분봉 ({next_candle_time}) 예측 결과:")
print(f"  - 상승/하락 예측: {trend}")


/home/amgos45/coin_transformer/amgos/lib/python3.8/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
/tmp/ipykernel_824211/331309710.py:58: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be l

다음 5분봉 (2024-12-18 16:30:00) 예측 결과:
  - 상승/하락 예측: 하락
